# Demo: Exploratory analysis for a reservoir
This demo shows how to load a CSV exported from InfluxDB and run the exploratory analysis utilities in `src/explore.py`.

The notebook demonstrates both the automatic lags selection for ACF/PACF and how to pass a custom number of lags to the analysis.

Steps performed in the notebook:
1. Load CSV using `load_time_series` (handles InfluxDB-style header rows).
2. Select the field to analyze (`reserva` or `porcentaje`).
3. Convert to numeric, align index, and drop missing values before calling `analyze`.
4. Run analysis with automatic lags (default) and with a custom lags value.

In [ ]:
# Cell 2: Import utilities and prepare environment
import sys
import os
sys.path.append(os.path.abspath('../src'))

from plot_single import load_time_series
from explore import analyze
import pandas as pd

# Path to sample CSV
csv_path = os.path.abspath('../data/test.csv')

# Load data (load_time_series will detect the second-row header produced by InfluxDB)
df = load_time_series(csv_path)

print('Available columns:', list(df.columns))
print('Date range:', df.index.min(), 'to', df.index.max())

# Choose the field to analyze
field = 'reserva'  # change to 'porcentaje' if desired

# Validate field exists
if field not in df.columns:
    raise SystemExit(f"Field '{field}' not found in CSV. Available: {', '.join(df.columns)}")

# Convert to numeric and align index
series = pd.to_numeric(df[field], errors='coerce')
series.index = df.index

# Drop missing values to avoid length/index mismatches
series = series.dropna()

print(f'Prepared series for {field}: {series.shape[0]} records')

# Run exploratory analysis with automatic lags selection (interactive display)
analyze(series, value_col=field)

In [ ]:
# Cell 3: Run analysis with custom lags and save the output
# Example: force 180 lags (approx. 6 months for daily data)
out_path = os.path.abspath('explore_lags_180.png')
print('Running analysis with custom lags=180 and saving to', out_path)

analyze(series, value_col=field, output=out_path, lags=180)
print('Saved:', out_path)